In [1]:
import dataset
import pandas as pd
import json
import time
import secrets
import datetime
from datetime import timedelta
import requests
import urllib.parse
import sqlite3

# import the class that ingests a 1% sampling of all tweets via the Twitter API 
from Sample_TwitterListener_FINAL import Sample

# import the class that ingests tweets based on the provided keyword(s)
from Filter_TwitterListener_FINAL import Filter

In [2]:
# retrieve secrets 
consumerKey = secrets.consumerKey
consumerSecret = secrets.consumerSecret
accessToken = secrets.accessToken
accessTokenSecret = secrets.accessTokenSecret

In [3]:
%%time

# code to use the listener to retrieve a random sampling of tweets 
# it will pull in ~1% of all tweets 
# language=en parameter is used to only pull in English language tweets 
# the inputted value will limit the number of tweets retrieved, otherwise the script would run indefinitely or 
# until it enough random connection issues occured that would lead to the script disconnecting. 


tweet_count = 0
tweet_count = int(input ("How large do you want your tweet sample size to be? "))
scraper = Sample(consumerKey, consumerSecret, accessToken, accessTokenSecret, tweet_count)
scraper.sample(languages=['en'])



How large do you want your tweet sample size to be? 40
It’s time to stop the insanity that is killing public education https://t.co/ZISEBaZkeG via @baptist_news
too bad the members don't handle their twitter account :/
@MrPeanut Possibly the most important poll of our lifetime.
#Vegetable #Gardening: March 1st can't come soon enough!
 
#VegetableGardening
 
https://t.co/lXirqtqKB6 https://t.co/fU1YbKub55


Stream connection closed by Twitter


stream disconnecting, desired sample size received
CPU times: user 1.74 s, sys: 101 ms, total: 1.84 s
Wall time: 15.6 s


In [4]:
%%time

# This section will pull in a real time filtered stream based on the inputted search terms 

try: 
    tweet_count = 0
    tweet_count = int(input ("How large do you want your tweet sample size to be? "))
    count = 0
    phrase_list = []
    phrases_count = int(input ("Please enter how many terms or phrases to filter by: "))
    while (count < phrases_count):
        term = input("Please enter keyword or hastag to search:" )
        phrase_list.append(term)
        count+=1

    scraper = Filter(consumerKey, consumerSecret, accessToken, accessTokenSecret, tweet_count, phrase_list)
    scraper.filter(track=phrase_list, languages=['en'])
    
except KeyboardInterrupt:
    print("Stopped.")

How large do you want your tweet sample size to be? 20
Please enter how many terms or phrases to filter by: 1
Please enter keyword or hastag to search:Super Bowl
@Tanuyak That's one more beer added to the #MountainOfBusch #Sweepstakes! Now that we've PEAKed your interest, be sure to enter every day leading up to the Super Bowl. For another chance to win something big, head to👇 https://t.co/5sKiSktp0u


Stream connection closed by Twitter


@IDLAAA That's one more beer added to the #MountainOfBusch #Sweepstakes! Now that we've PEAKed your interest, be sure to enter every day leading up to the Super Bowl. For another chance to win something big, head to👇 https://t.co/5sKiSktp0u
stream disconnecting, desired sample size received
CPU times: user 2.08 s, sys: 193 ms, total: 2.27 s
Wall time: 54.5 s


In [5]:
%%time 

#quick look at the DB To see how many records there are

# query DB being populated by Twitter listener 
con = sqlite3.connect("filterTweets.db")

# select the entire table 
df = pd.read_sql_query("SELECT * from filter_tweet_table", con)

# close the connection 
con.close()


print(len(df))

20
CPU times: user 3.6 ms, sys: 2.4 ms, total: 6 ms
Wall time: 4.88 ms


In [6]:
df.head(50)


,id,text,timeStamp,verified_user,followers,retweetCount,favoriteCount,location,sentiment,subjectivity,score
0,1,The Super Bowl is often the most exciting nigh...,2022-02-13 02:14:38.000000,1,70266,0,0,None,0.377778,0.655556,"{""neg"": 0.0, ""neu"": 0.774, ""pos"": 0.226, ""comp..."
1,2,I cant wait to go to the gym tomorrow cause it...,2022-02-13 02:14:40.000000,0,331,0,0,None,0.066667,0.533333,"{""neg"": 0.152, ""neu"": 0.709, ""pos"": 0.138, ""co..."
2,3,Twins Nikki and Brie Bella match in red as the...,2022-02-13 02:14:40.000000,0,247,0,0,None,0.166667,0.333333,"{""neg"": 0.0, ""neu"": 0.708, ""pos"": 0.292, ""comp..."
3,4,Probable scenario: FTX has an advertisement d...,2022-02-13 02:14:42.000000,0,437,0,0,None,0.229437,0.594949,"{""neg"": 0.0, ""neu"": 0.737, ""pos"": 0.263, ""comp..."
4,5,Fran’s Favorites: Ohio’s first lady shares Sup...,2022-02-13 02:14:42.000000,1,245717,0,0,None,0.291667,0.500000,"{""neg"": 0.0, ""neu"": 0.473, ""pos"": 0.527, ""comp..."
5,6,The NFL would like to remind everyone that the...,2022-02-13 02:14:51.000000,0,3481,0,0,None,0.333333,0.666667,"{""neg"": 0.067, ""neu"": 0.736, ""pos"": 0.196, ""co..."
6,7,UFC 271 tonight and the Super Bowl tomorrow? T...,2022-02-13 02:14:52.000000,0,8039,0,0,None,0.333333,0.666667,"{""neg"": 0.0, ""neu"": 0.625, ""pos"": 0.375, ""comp..."
7,8,I’m scrolling through Twitter on Super Bowl Ev...,2022-02-13 02:14:52.000000,0,603,0,0,None,0.144444,0.522222,"{""neg"": 0.0, ""neu"": 0.747, ""pos"": 0.253, ""comp..."
8,9,Street Artist Sabo Puts Up Recall Soros-Backed...,2022-02-13 02:14:53.000000,0,10,0,0,None,0.333333,0.666667,"{""neg"": 0.0, ""neu"": 0.83, ""pos"": 0.17, ""compou..."
9,10,@Tanuyak That's one more beer added to the #Mo...,2022-02-13 02:14:54.000000,1,157183,0,0,None,0.451667,0.511111,"{""neg"": 0.0, ""neu"": 0.683, ""pos"": 0.317, ""comp..."
